In [ ]:
import numpy as np
from scipy.stats import norm
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# ======================
# PCA 由来の損失変数
# ======================
Z = np.array([0.791, 0.275, 0.438, -0.370, 0.073, -0.201, -1.006])
L = -Z  # 損失なので符号反転

mu_L = L.mean()
sigma_L = L.std(ddof=0)

# ======================
# 95% VaR（古典計算）
# ======================
alpha = 0.95  # 上位5%のVaR
l_var95 = norm.ppf(alpha, mu_L, sigma_L)
print("95% VaR (classical) =", l_var95)

95% VaR (classical) = 0.8983060973819932


In [ ]:
# ======================
# 量子回路によるCDF埋め込み
# ======================
# 95%に対応する振幅
p_target = alpha
theta = 2 * np.arcsin(np.sqrt(p_target))

qc = QuantumCircuit(1, 1)
qc.ry(theta, 0)
qc.measure(0, 0)

# 実行
backend = AerSimulator()
shots = 10000
tqc = transpile(qc, backend)
job = backend.run(tqc, shots=shots)
result = job.result()
counts = result.get_counts()
p_quantum = counts.get("1", 0) / shots

print("Quantum estimated 95% CDF =", p_quantum)

Quantum estimated 95% CDF = 0.949


In [ ]:
import numpy as np
from scipy.stats import norm
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

# ======================
# PCA 由来の損失変数
# ======================
Z = np.array([0.791, 0.275, 0.438, -0.370, 0.073, -0.201, -1.006])
L = -Z  # 損失なので符号反転

mu_L = L.mean()
sigma_L = L.std(ddof=0)

# ======================
# 95% VaR（古典計算）
# ======================
alpha = 0.95  # 上位5%のVaR
l_var95_classical = norm.ppf(alpha, mu_L, sigma_L)
print("95% VaR (classical) =", l_var95_classical)

# ======================
# 量子回路によるCDF埋め込み
# ======================
p_target = alpha
theta = 2 * np.arcsin(np.sqrt(p_target))

qc = QuantumCircuit(1, 1)
qc.ry(theta, 0)
qc.measure(0, 0)

# 実行
backend = AerSimulator()
shots = 10000
tqc = transpile(qc, backend)
job = backend.run(tqc, shots=shots)
result = job.result()
counts = result.get_counts()
p_quantum = counts.get("1", 0) / shots
print("Quantum estimated 95% CDF =", p_quantum)

# ======================
# 量子測定結果から損失値を逆算
# ======================
l_var95_quantum = norm.ppf(p_quantum, mu_L, sigma_L)
print("Quantum-based VaR =", l_var95_quantum)


95% VaR (classical) = 0.8983060973819932
Quantum estimated 95% CDF = 0.9496
Quantum-based VaR = 0.8961947106114323


In [ ]:
import numpy as np

# 元データ
R = np.array([
    [0.027, -0.046, -0.377],
    [0.001,  0.034,  0.207],
    [0.01,  -0.006,  0.032],
    [0.049,  0.333,  0.787],
    [0.051,  0.34,   0.243],
    [0.47,   0.246,  0.527],
    [0.367,  0.918,  1.102]
])

# 第1主成分スコア
Z = np.array([0.791, 0.275, 0.438, -0.370, 0.073, -0.201, -1.006])

# 元データの総分散（全要素の分散）
total_var = np.sum(np.var(R, axis=0, ddof=1))

# 第1主成分の分散
pc1_var = np.var(Z, ddof=1)

# 寄与率
explained_ratio = pc1_var / total_var

print("第1主成分の寄与率:", explained_ratio)


第1主成分の寄与率: 0.8942900763358779


In [ ]:
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
import numpy as np

def build_var_oracle_circuit(x_threshold):
    p = 0.1587
    theta = 2 * np.arcsin(np.sqrt(p))

    qc = QuantumCircuit(3)

    # PC1, PC2 の確率エンコード
    qc.ry(theta, 0)
    qc.ry(theta, 1)

    # ここでは例として固定値（実際は計算済みの値を代入）
    L_01 = 1.0
    L_10 = 2.0
    L_11 = 3.0

    # オラクル
    if L_01 > x_threshold:
        qc.ch(1, 2)
    if L_10 > x_threshold:
        qc.ch(0, 2)
    if L_11 > x_threshold:
        qc.ccx(0, 1, 2)

    return qc

# 回路生成
qc = build_var_oracle_circuit(x_threshold=1.5)

# テキスト表示
print(qc.draw())

# Jupyter Notebook 用（図として表示）
# circuit_drawer(qc, output="mpl")